In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import seaborn as sns
import os 
import warnings
from patsy import dmatrices
import statsmodels.api as sm;
from statsmodels.stats.outliers_influence import variance_inflation_factor
from pylab import rcParams

%matplotlib inline


import lightgbm as lgb
import sklearn

from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import shuffle
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report,accuracy_score
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from sklearn.model_selection import StratifiedKFold,KFold
from sklearn.model_selection import train_test_split


import warnings

Using TensorFlow backend.


In [2]:
#data = pd.read_csv('D:/FV_raw/S01_total_rev.csv')
data = pd.read_csv('C:\\Users\\\koreaw\\비즈니스어낼리틱스\\BA_data\\S01_data_rev_1.csv')
print(data.shape)

(53692, 94)


# EDA

In [3]:
var = data.drop(['L0t','Class','S_NO','D_ID','FV89','FV90'], axis=1)
target = data['Class']

min_max_scaler = MinMaxScaler()
data2_norm=min_max_scaler.fit_transform(var)
data2_norm = pd.DataFrame(data2_norm)
data_norm = data2_norm.merge(target.to_frame(),left_index=True, right_index = True)
data_norm.head(5)

,0,1,2,3,4,5,6,7,8,9,...,79,80,81,82,83,84,85,86,87,Class
0,0.704672,0.613381,0.602628,0.3787,0.0,0.613485,0.826649,0.826649,0.6131,0.0,...,0.085121,0.036100,0.088404,0.135428,0.0328,0.7236,0.00000,0.000000,0.5,17
1,0.276039,0.489615,0.488530,0.7500,0.0,0.489699,0.556513,0.556513,0.3868,0.0,...,0.099081,0.023713,0.078263,0.139745,0.0347,0.7902,0.00000,0.003934,1.0,21
2,0.180883,0.376915,0.429988,0.7500,0.0,0.376979,0.512430,0.396420,0.3868,0.0,...,0.115764,0.156609,0.222002,0.141652,0.1174,0.2804,0.01976,0.023164,1.0,9
3,0.227390,0.416411,0.446834,0.7901,0.0,0.416482,0.512430,0.460225,0.3868,0.0,...,0.090398,0.067953,0.152116,0.118261,0.0518,0.6053,0.00000,0.002404,1.0,21
4,0.161380,0.365169,0.429988,0.6875,0.0,0.365231,0.460225,0.460225,0.3868,0.0,...,0.009193,0.017342,0.024471,0.115149,0.0054,0.9704,0.00000,0.000000,0.5,9


In [4]:
pca = PCA(n_components=0.95) #95%이상의 분산을 설명력을 갖는 차원축소
new_data2 = pca.fit_transform(data2_norm)
data2_norm2 = pd.DataFrame(new_data2)
data_norm = data2_norm2.merge(target.to_frame(),left_index=True, right_index = True)
data_norm.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,Class
0,0.130768,-1.038052,0.697290,0.165806,0.487045,0.896660,-0.127558,0.114706,0.154536,0.349317,0.264798,0.195927,0.076353,0.145501,-0.127520,-0.110704,0.127078,0.020761,17
1,-0.249376,-0.617195,-0.025617,-0.003057,-0.323223,-0.159566,-0.084079,0.192891,0.120966,0.012226,-0.033878,-0.075836,0.219057,0.133422,0.037402,-0.105950,0.002519,0.057020,21
2,-0.269553,-0.492975,-0.262176,-0.183749,-0.594876,0.122842,-0.117533,0.290239,-0.408543,-0.068274,-0.084349,-0.002861,-0.122769,0.042745,-0.005036,0.147850,-0.101937,-0.093063,9
3,-0.399706,-0.494862,-0.085869,-0.167540,0.414870,0.053105,-0.380552,0.034769,-0.154443,-0.006329,-0.109446,0.142629,-0.130427,0.049213,0.018222,-0.025605,-0.074968,-0.057281,21
4,-0.791584,-0.367674,-0.300121,0.297141,0.648399,0.088933,0.139348,-0.035591,0.070308,-0.099122,-0.016146,-0.236745,-0.078888,0.190031,0.224928,-0.088945,0.091973,-0.054262,9


## Class 7개, 최종 데이터 셋 

In [5]:
def func(data_norm) :
    if data_norm['Class'] == 9 or data_norm['Class'] == 21 or data_norm['Class'] == 17 \
    or data_norm['Class'] == 16 or data_norm['Class'] == 198 or data_norm['Class'] == 76 \
    or data_norm['Class'] == 22 :
        return data_norm['Class']
    else : return 200
data_norm['ClassA'] = data_norm.apply(func, axis = 1)
data_norm1 = data_norm[data_norm['ClassA'] != 200]

data_norm1.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,Class,ClassA
0,0.130768,-1.038052,0.697290,0.165806,0.487045,0.896660,-0.127558,0.114706,0.154536,0.349317,0.264798,0.195927,0.076353,0.145501,-0.127520,-0.110704,0.127078,0.020761,17,17.0
1,-0.249376,-0.617195,-0.025617,-0.003057,-0.323223,-0.159566,-0.084079,0.192891,0.120966,0.012226,-0.033878,-0.075836,0.219057,0.133422,0.037402,-0.105950,0.002519,0.057020,21,21.0
2,-0.269553,-0.492975,-0.262176,-0.183749,-0.594876,0.122842,-0.117533,0.290239,-0.408543,-0.068274,-0.084349,-0.002861,-0.122769,0.042745,-0.005036,0.147850,-0.101937,-0.093063,9,9.0
3,-0.399706,-0.494862,-0.085869,-0.167540,0.414870,0.053105,-0.380552,0.034769,-0.154443,-0.006329,-0.109446,0.142629,-0.130427,0.049213,0.018222,-0.025605,-0.074968,-0.057281,21,21.0
4,-0.791584,-0.367674,-0.300121,0.297141,0.648399,0.088933,0.139348,-0.035591,0.070308,-0.099122,-0.016146,-0.236745,-0.078888,0.190031,0.224928,-0.088945,0.091973,-0.054262,9,9.0


In [6]:
train, test = train_test_split(data_norm1, test_size = 0.3)
x_train = train.drop(['Class','ClassA'], axis=1)
y_train = train['Class']
x_test = test.drop(['Class','ClassA'], axis=1)
y_test = test['Class']

ros = RandomOverSampler(random_state=0)
x_train_res, y_train_res = ros.fit_resample(x_train, y_train)

x_train_res = np.array(x_train_res)
y_train_res = np.array(y_train_res)

log_model = LogisticRegression(solver='liblinear')
log_model.fit(x_train_res, y_train_res)
pred_lm = log_model.predict(x_test)
accuracy = accuracy_score(y_test, pred_lm)
print(classification_report(y_test, pred_lm))

from sklearn import svm
sv_model = svm.SVC(gamma = 'scale')
sv_model.fit(x_train_res, y_train_res)
pred_sv = sv_model.predict(x_test)
print(classification_report(y_test, pred_sv))

from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=300, max_depth=30, random_state =0)
rf_model.fit(x_train_res, y_train_res)
pred_rf = rf_model.predict(x_test)
print(classification_report(y_test, pred_rf))

from sklearn.neural_network import MLPClassifier
nn_model = MLPClassifier(max_iter=10000, 
                      verbose=True, 
                      hidden_layer_sizes=(300,100), 
                      activation = 'relu',
                      alpha = 1e-5,
                      beta_1=0.9,
                      beta_2=0.999,
                      power_t = 0.5)
nn_model.fit(x_train_res, y_train_res)
pred_nn = nn_model.predict(x_test)
print(classification_report(y_test, pred_nn))

C:\ProgramData\Anaconda3\envs\tf_2\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


              precision    recall  f1-score   support

           9       0.96      0.50      0.66      8814
          16       0.39      0.40      0.39       740
          17       0.22      0.23      0.23       978
          21       0.36      0.56      0.44      1006
          22       0.37      0.60      0.46       505
          76       0.22      0.76      0.34       592
         198       0.21      0.87      0.33       562

    accuracy                           0.51     13197
   macro avg       0.39      0.56      0.41     13197
weighted avg       0.74      0.51      0.56     13197

              precision    recall  f1-score   support

           9       0.97      0.64      0.77      8814
          16       0.48      0.59      0.53       740
          17       0.32      0.43      0.36       978
          21       0.59      0.65      0.62      1006
          22       0.49      0.57      0.53       505
          76       0.31      0.71      0.43       592
         198       0.24 

In [7]:
def func(data_norm1) :
    if data_norm1['Class'] == 9 : return 0
    else : return 1
data_norm1['ClassA'] = data_norm1.apply(func, axis = 1)

train, test = train_test_split(data_norm1, test_size = 0.3)
x_train = train.drop(['Class','ClassA'], axis=1)
y_train = train['ClassA']
x_test = test.drop(['Class','ClassA'], axis=1)
y_test = test['ClassA']

smote = SMOTE(ratio='minority')
x_train_sm, y_train_sm = smote.fit_sample(x_train, y_train)

x_train_sm = np.array(x_train_sm)
y_train_sm = np.array(y_train_sm)

log_model = LogisticRegression(solver='liblinear')
log_model.fit(x_train_sm, y_train_sm)
pred_lm_sm = log_model.predict(x_test)
print("LogisticRegression ")
print(classification_report(y_test, pred_lm_sm))

from sklearn import svm
sv_model = svm.SVC(gamma = 'scale')
sv_model.fit(x_train_sm, y_train_sm)
pred_sv_sm = sv_model.predict(x_test)
print("SVM")
print(classification_report(y_test, pred_sv_sm, target_names=['class 0', 'class 1']))

from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=300, max_depth=30, random_state =0)
rf_model.fit(x_train_sm, y_train_sm)
pred_rf_sm = rf_model.predict(x_test)
print("RandomForest")
print(classification_report(y_test, pred_rf_sm, target_names=['class 0', 'class 1']))

from sklearn.neural_network import MLPClassifier
nn_model = MLPClassifier(max_iter=10000, 
                      verbose=True, 
                      hidden_layer_sizes=(300,100), 
                      activation = 'relu',
                      alpha = 1e-5,
                      beta_1=0.9,
                      beta_2=0.999,
                      power_t = 0.5)
nn_model.fit(x_train_sm, y_train_sm)
pred_nn_sm = nn_model.predict(x_test)
print("Neural Network")
print(classification_report(y_test, pred_nn_sm, target_names=['class 0', 'class 1']))

C:\ProgramData\Anaconda3\envs\tf_2\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


LogisticRegression 
              precision    recall  f1-score   support

           0       0.88      0.81      0.85      8863
           1       0.67      0.78      0.72      4334

    accuracy                           0.80     13197
   macro avg       0.78      0.80      0.78     13197
weighted avg       0.81      0.80      0.80     13197

SVM
              precision    recall  f1-score   support

     class 0       0.91      0.87      0.89      8863
     class 1       0.75      0.83      0.79      4334

    accuracy                           0.86     13197
   macro avg       0.83      0.85      0.84     13197
weighted avg       0.86      0.86      0.86     13197

RandomForest
              precision    recall  f1-score   support

     class 0       0.89      0.90      0.90      8863
     class 1       0.79      0.78      0.79      4334

    accuracy                           0.86     13197
   macro avg       0.84      0.84      0.84     13197
weighted avg       0.86      0.86    

In [8]:
data_norm2 = data_norm1[data_norm1['ClassA']==1]

def func(data_norm2) :
    if data_norm2['Class'] == 21 : return 0
    else : return 1
data_norm2['ClassA'] = data_norm2.apply(func, axis = 1)

train, test = train_test_split(data_norm2, test_size = 0.3)
x_train = train.drop(['Class','ClassA'], axis=1)
y_train = train['ClassA']
x_test = test.drop(['Class','ClassA'], axis=1)
y_test = test['ClassA']

smote = SMOTE(ratio='minority')
x_train_sm, y_train_sm = smote.fit_sample(x_train, y_train)

x_train_sm = np.array(x_train_sm)
y_train_sm = np.array(y_train_sm)

print("LogisticRegression ")

log_model = LogisticRegression(solver='liblinear')
log_model.fit(x_train_sm, y_train_sm)
pred_lm_sm = log_model.predict(x_test)
print(classification_report(y_test, pred_lm_sm))

from sklearn import svm
sv_model = svm.SVC(gamma = 'scale')
sv_model.fit(x_train_sm, y_train_sm)
pred_sv_sm = sv_model.predict(x_test)
print("SVM")

print(classification_report(y_test, pred_sv_sm, target_names=['class 0', 'class 1']))

from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=300, max_depth=30, random_state =0)
rf_model.fit(x_train_sm, y_train_sm)
pred_rf_sm = rf_model.predict(x_test)
print("RandomForest")
print(classification_report(y_test, pred_rf_sm, target_names=['class 0', 'class 1']))

from sklearn.neural_network import MLPClassifier
nn_model = MLPClassifier(max_iter=10000, 
                      verbose=True, 
                      hidden_layer_sizes=(300,100), 
                      activation = 'relu',
                      alpha = 1e-5,
                      beta_1=0.9,
                      beta_2=0.999,
                      power_t = 0.5)
nn_model.fit(x_train_sm, y_train_sm)
pred_nn_sm = nn_model.predict(x_test)
print("Neural Network")
print(classification_report(y_test, pred_nn_sm, target_names=['class 0', 'class 1']))

C:\ProgramData\Anaconda3\envs\tf_2\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


LogisticRegression 
              precision    recall  f1-score   support

           0       0.47      0.73      0.57       992
           1       0.90      0.76      0.83      3334

    accuracy                           0.75      4326
   macro avg       0.69      0.74      0.70      4326
weighted avg       0.81      0.75      0.77      4326

SVM
              precision    recall  f1-score   support

     class 0       0.66      0.79      0.72       992
     class 1       0.93      0.88      0.91      3334

    accuracy                           0.86      4326
   macro avg       0.80      0.84      0.81      4326
weighted avg       0.87      0.86      0.86      4326

RandomForest
              precision    recall  f1-score   support

     class 0       0.73      0.71      0.72       992
     class 1       0.92      0.92      0.92      3334

    accuracy                           0.88      4326
   macro avg       0.83      0.82      0.82      4326
weighted avg       0.87      0.88    

In [9]:
data_norm3 = data_norm2[data_norm2['ClassA']==1]

def func(data_norm3) :
    if data_norm3['Class'] == 17 : return 0
    else : return 1
data_norm3['ClassA'] = data_norm3.apply(func, axis = 1)

train, test = train_test_split(data_norm3, test_size = 0.3)
x_train = train.drop(['Class','ClassA'], axis=1)
y_train = train['ClassA']
x_test = test.drop(['Class','ClassA'], axis=1)
y_test = test['ClassA']

smote = SMOTE(ratio='minority')
x_train_sm, y_train_sm = smote.fit_sample(x_train, y_train)

x_train_sm = np.array(x_train_sm)
y_train_sm = np.array(y_train_sm)

log_model = LogisticRegression(solver='liblinear')
log_model.fit(x_train_sm, y_train_sm)
pred_lm_sm = log_model.predict(x_test)
print("LogisticRegression ")

print(classification_report(y_test, pred_lm_sm))

from sklearn import svm
sv_model = svm.SVC(gamma = 'scale')
sv_model.fit(x_train_sm, y_train_sm)
pred_sv_sm = sv_model.predict(x_test)
print("SVM")

print(classification_report(y_test, pred_sv_sm, target_names=['class 0', 'class 1']))

from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=300, max_depth=30, random_state =0)
rf_model.fit(x_train_sm, y_train_sm)
pred_rf_sm = rf_model.predict(x_test)
print("RandomForest")

print(classification_report(y_test, pred_rf_sm, target_names=['class 0', 'class 1']))

from sklearn.neural_network import MLPClassifier
nn_model = MLPClassifier(max_iter=10000, 
                      verbose=True, 
                      hidden_layer_sizes=(300,100), 
                      activation = 'relu',
                      alpha = 1e-5,
                      beta_1=0.9,
                      beta_2=0.999,
                      power_t = 0.5)
nn_model.fit(x_train_sm, y_train_sm)
pred_nn_sm = nn_model.predict(x_test)
print("Neural Network")

print(classification_report(y_test, pred_nn_sm, target_names=['class 0', 'class 1']))

C:\ProgramData\Anaconda3\envs\tf_2\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


LogisticRegression 
              precision    recall  f1-score   support

           0       0.43      0.69      0.53       955
           1       0.84      0.64      0.72      2401

    accuracy                           0.65      3356
   macro avg       0.63      0.66      0.63      3356
weighted avg       0.72      0.65      0.67      3356

SVM
              precision    recall  f1-score   support

     class 0       0.53      0.76      0.62       955
     class 1       0.88      0.73      0.80      2401

    accuracy                           0.74      3356
   macro avg       0.71      0.74      0.71      3356
weighted avg       0.78      0.74      0.75      3356

RandomForest
              precision    recall  f1-score   support

     class 0       0.59      0.60      0.60       955
     class 1       0.84      0.83      0.84      2401

    accuracy                           0.77      3356
   macro avg       0.71      0.72      0.72      3356
weighted avg       0.77      0.77    

In [10]:
data_norm4 = data_norm3[data_norm3['ClassA']==1]

def func(data_norm4) :
    if data_norm4['Class'] == 16 : return 0
    else : return 1
data_norm4['ClassA'] = data_norm4.apply(func, axis = 1)

train, test = train_test_split(data_norm4, test_size = 0.3)
x_train = train.drop(['Class','ClassA'], axis=1)
y_train = train['ClassA']
x_test = test.drop(['Class','ClassA'], axis=1)
y_test = test['ClassA']

smote = SMOTE(ratio='minority')
x_train_sm, y_train_sm = smote.fit_sample(x_train, y_train)

x_train_sm = np.array(x_train_sm)
y_train_sm = np.array(y_train_sm)

log_model = LogisticRegression(solver='liblinear')
log_model.fit(x_train_sm, y_train_sm)
pred_lm_sm = log_model.predict(x_test)
print("LogisticRegression ")

print(classification_report(y_test, pred_lm_sm))

from sklearn import svm
sv_model = svm.SVC(gamma = 'scale')
sv_model.fit(x_train_sm, y_train_sm)
pred_sv_sm = sv_model.predict(x_test)
print("SVM")

print(classification_report(y_test, pred_sv_sm, target_names=['class 0', 'class 1']))

from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=300, max_depth=30, random_state =0)
rf_model.fit(x_train_sm, y_train_sm)
pred_rf_sm = rf_model.predict(x_test)
print("RandomForest")

print(classification_report(y_test, pred_rf_sm, target_names=['class 0', 'class 1']))

from sklearn.neural_network import MLPClassifier
nn_model = MLPClassifier(max_iter=10000, 
                      verbose=True, 
                      hidden_layer_sizes=(300,100), 
                      activation = 'relu',
                      alpha = 1e-5,
                      beta_1=0.9,
                      beta_2=0.999,
                      power_t = 0.5)
nn_model.fit(x_train_sm, y_train_sm)
pred_nn_sm = nn_model.predict(x_test)
print("Neural Network")

print(classification_report(y_test, pred_nn_sm, target_names=['class 0', 'class 1']))

LogisticRegression 

C:\ProgramData\Anaconda3\envs\tf_2\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  



              precision    recall  f1-score   support

           0       0.53      0.78      0.63       748
           1       0.87      0.69      0.77      1650

    accuracy                           0.72      2398
   macro avg       0.70      0.74      0.70      2398
weighted avg       0.77      0.72      0.73      2398

SVM
              precision    recall  f1-score   support

     class 0       0.61      0.84      0.71       748
     class 1       0.91      0.76      0.83      1650

    accuracy                           0.78      2398
   macro avg       0.76      0.80      0.77      2398
weighted avg       0.82      0.78      0.79      2398

RandomForest
              precision    recall  f1-score   support

     class 0       0.67      0.72      0.69       748
     class 1       0.87      0.84      0.85      1650

    accuracy                           0.80      2398
   macro avg       0.77      0.78      0.77      2398
weighted avg       0.80      0.80      0.80      2398

I

In [11]:
data_norm5 = data_norm4[data_norm4['ClassA']==1]

def func(data_norm5) :
    if data_norm5['Class'] == 198 : return 0
    else : return 1
data_norm5['ClassA'] = data_norm5.apply(func, axis = 1)

train, test = train_test_split(data_norm5, test_size = 0.3)
x_train = train.drop(['Class','ClassA'], axis=1)
y_train = train['ClassA']
x_test = test.drop(['Class','ClassA'], axis=1)
y_test = test['ClassA']

smote = SMOTE(ratio='minority')
x_train_sm, y_train_sm = smote.fit_sample(x_train, y_train)

x_train_sm = np.array(x_train_sm)
y_train_sm = np.array(y_train_sm)

log_model = LogisticRegression(solver='liblinear')
log_model.fit(x_train_sm, y_train_sm)
pred_lm_sm = log_model.predict(x_test)
print("LogisticRegression ")

print(classification_report(y_test, pred_lm_sm))

from sklearn import svm
sv_model = svm.SVC(gamma = 'scale')
sv_model.fit(x_train_sm, y_train_sm)
pred_sv_sm = sv_model.predict(x_test)
print("SVM")

print(classification_report(y_test, pred_sv_sm, target_names=['class 0', 'class 1']))

from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=300, max_depth=30, random_state =0)
rf_model.fit(x_train_sm, y_train_sm)
pred_rf_sm = rf_model.predict(x_test)
print("RandomForest")

print(classification_report(y_test, pred_rf_sm, target_names=['class 0', 'class 1']))

from sklearn.neural_network import MLPClassifier
nn_model = MLPClassifier(max_iter=10000, 
                      verbose=True, 
                      hidden_layer_sizes=(300,100), 
                      activation = 'relu',
                      alpha = 1e-5,
                      beta_1=0.9,
                      beta_2=0.999,
                      power_t = 0.5)
nn_model.fit(x_train_sm, y_train_sm)
pred_nn_sm = nn_model.predict(x_test)
print("Neural Network")

print(classification_report(y_test, pred_nn_sm, target_names=['class 0', 'class 1']))

C:\ProgramData\Anaconda3\envs\tf_2\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


LogisticRegression 
              precision    recall  f1-score   support

           0       0.96      0.98      0.97       595
           1       0.99      0.98      0.98      1070

    accuracy                           0.98      1665
   macro avg       0.98      0.98      0.98      1665
weighted avg       0.98      0.98      0.98      1665

SVM
              precision    recall  f1-score   support

     class 0       0.98      0.99      0.98       595
     class 1       0.99      0.99      0.99      1070

    accuracy                           0.99      1665
   macro avg       0.99      0.99      0.99      1665
weighted avg       0.99      0.99      0.99      1665

RandomForest
              precision    recall  f1-score   support

     class 0       0.99      0.98      0.99       595
     class 1       0.99      1.00      0.99      1070

    accuracy                           0.99      1665
   macro avg       0.99      0.99      0.99      1665
weighted avg       0.99      0.99    

In [12]:
data_norm6 = data_norm5[data_norm5['ClassA']==1]

def func(data_norm6) :
    if data_norm6['Class'] == 76 : return 0
    else : return 1
data_norm6['ClassA'] = data_norm6.apply(func, axis = 1)

train, test = train_test_split(data_norm6, test_size = 0.3)
x_train = train.drop(['Class','ClassA'], axis=1)
y_train = train['ClassA']
x_test = test.drop(['Class','ClassA'], axis=1)
y_test = test['ClassA']

smote = SMOTE(ratio='minority')
x_train_sm, y_train_sm = smote.fit_sample(x_train, y_train)

x_train_sm = np.array(x_train_sm)
y_train_sm = np.array(y_train_sm)

log_model = LogisticRegression(solver='liblinear')
log_model.fit(x_train_sm, y_train_sm)
pred_lm_sm = log_model.predict(x_test)
print("LogisticRegression ")

print(classification_report(y_test, pred_lm_sm))

from sklearn import svm
sv_model = svm.SVC(gamma = 'scale')
sv_model.fit(x_train_sm, y_train_sm)
pred_sv_sm = sv_model.predict(x_test)
print("SVM")

print(classification_report(y_test, pred_sv_sm, target_names=['class 0', 'class 1']))

from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=300, max_depth=30, random_state =0)
rf_model.fit(x_train_sm, y_train_sm)
pred_rf_sm = rf_model.predict(x_test)
print("RandomForest")

print(classification_report(y_test, pred_rf_sm, target_names=['class 0', 'class 1']))

from sklearn.neural_network import MLPClassifier
nn_model = MLPClassifier(max_iter=10000, 
                      verbose=True, 
                      hidden_layer_sizes=(300,100), 
                      activation = 'relu',
                      alpha = 1e-5,
                      beta_1=0.9,
                      beta_2=0.999,
                      power_t = 0.5)
nn_model.fit(x_train_sm, y_train_sm)
pred_nn_sm = nn_model.predict(x_test)
print("Neural Network")

print(classification_report(y_test, pred_nn_sm, target_names=['class 0', 'class 1']))

LogisticRegression 
              precision    recall  f1-score   support

           0       0.88      0.90      0.89       524
           1       0.90      0.88      0.89       527

    accuracy                           0.89      1051
   macro avg       0.89      0.89      0.89      1051
weighted avg       0.89      0.89      0.89      1051



C:\ProgramData\Anaconda3\envs\tf_2\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


SVM
              precision    recall  f1-score   support

     class 0       0.89      0.90      0.89       524
     class 1       0.90      0.88      0.89       527

    accuracy                           0.89      1051
   macro avg       0.89      0.89      0.89      1051
weighted avg       0.89      0.89      0.89      1051

RandomForest
              precision    recall  f1-score   support

     class 0       0.90      0.90      0.90       524
     class 1       0.90      0.90      0.90       527

    accuracy                           0.90      1051
   macro avg       0.90      0.90      0.90      1051
weighted avg       0.90      0.90      0.90      1051

Iteration 1, loss = 0.61050139
Iteration 2, loss = 0.45701077
Iteration 3, loss = 0.35907046
Iteration 4, loss = 0.32402408
Iteration 5, loss = 0.30304392
Iteration 6, loss = 0.28751480
Iteration 7, loss = 0.27941593
Iteration 8, loss = 0.27327380
Iteration 9, loss = 0.26642807
Iteration 10, loss = 0.26461112
Iteration 11, loss